# Esercitazione 6c - Classificazione basicness di synsets

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

*Consegna*: classificazione basic/advanced. Si richiedere l'uso (o meno) del dataset su basicness per fare classificazione automatica (binaria, basic/advanced) su nuovi termini e/o synset presi in esame.

Come primo passo scarichiamo e decomprimiamo il dataset ottenuto da Moodle.

In [ ]:
# Open the inspect element into your moodle session, then paste the "MoodleSession" field value in the storage/cookies tab
moodle_session_cookie = 'pl0bqb83f3lk6so8g6oo6b5bkq'

!curl --cookie 'MoodleSession={moodle_session_cookie}' "https://informatica.i-learn.unito.it/pluginfile.php/366022/mod_folder/content/0/dataset_basic_advanced_TLN2023.zip?forcedownload=1" -o dataset.zip
!unzip -q dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  239k  100  239k    0     0   188k      0  0:00:01  0:00:01 --:--:--  188k


Il dataset scaricato è composto da 10 files in formato `.json`, ognuno contenente il risultato di un esperimento in cui a dei respondents veniva richiesto di decidere se una serie di synsets dati fosse *basic* oppure *advanced*. I synsets presi in considerazione sono i medesimi per ogni esperimento.

Ai fini del caricamento del dataset è necessario trovare un modo per aggregare i vari esperimenti in un singolo dataset. Il criterio di decisione per scegliere se un dato synset è da considerarsi *basic* o *advanced* è quello del voto di maggioranza: se la maggior parte dei respondents sui $10$ esperimenti ha determinato che un termine era *basic*/*advanced* allora questo termine verrà considerato *basic*/*advanced* nel dataset aggregato.

Dal punto di vista implementativo, vengono contate il numero di volte che un synset è stato considerato *basic* per poi dividerlo per il numero di esperimenti totali ($10$). Se il risultato rappresenta almeno il 50% dei respondents ($>.5$) allora si tratta di un termine *basic* ($1$), altrimenti sarà considerato *advanced* ($0$).

In [ ]:
from typing_extensions import dataclass_transform
import pandas as pd
from nltk.corpus import wordnet as wn
import os
import json
import re
import nltk
nltk.download('wordnet')

# Given a row in the given dataset, transforms it into a wordnet synset
row_to_synset = lambda row: wn.synset(re.search(r'\((.*?)\)', row).group(1).strip("'"))

# Define a function that given a synset returns its name (primary lemma)
# separated by spaces (instead of being represented in snake_case)
synset_name = lambda syn: syn.lemma_names()[0].replace('_', ' ')

# Aggregate provided datasets while adding a column with wordfreq
def load_dataset(dataset_path):
  dataset = pd.DataFrame()
  filenames = os.listdir(dataset_path)
  n_files = len(filenames)
  for filename in filenames:
    with open(f'{dataset_path}/{filename}') as ds_file:
      current_dataset = json.load(ds_file)
      # Initialize dataset
      if dataset.empty:
        dataset['synsets'] = [row_to_synset(row) for row in current_dataset['dataset']]
        dataset['is_basic'] = 0

      dataset['is_basic'] += [1 if row == 'basic' else 0 for row in current_dataset['answers']]

  # Get the average (over all files) then discretize the value
  dataset['is_basic'] = (dataset['is_basic'] / n_files).map(lambda x: 1 if x > .5 else 0)
  return dataset

dataset = load_dataset('./dataset_basic_advanced_TLN2023')
dataset

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,synsets,is_basic
0,Synset('war.n.01'),1
1,Synset('fiefdom.n.01'),0
2,Synset('bed.n.03'),1
3,Synset('return_on_invested_capital.n.01'),0
4,Synset('texture.n.02'),1
...,...,...
499,Synset('reading.n.03'),1
500,Synset('sanctimoniousness.n.01'),0
501,Synset('chalcedony.n.01'),0
502,Synset('stopcock.n.01'),0


## Handmade Features

Il primo approccio valutato è stato quello di cercare delle features che potessero essere significative ai fini della classificazione. In questo caso, sono state individuate tre features che potrebbero essere determinanti in merito alla *basicness* di un synset:

* La lunghezza di un termine;
* La frequenza (globale) con cui questo termine viene utilizzato;
* La profondità massima nella multi-gerarchia di WordNet.

La prima è giustificata dal fatto che spesso termini molto complessi tendono ad essere molto lunghi (anche se non si tratta di una regola generale), mentre la seconda perché ragionevolmente un termine non di base (*advanced*) sarà utilizzato di rado in una lingua. Si pensi, per esempio, a termini tecnici che vengono utilizzati solamente nella loro letteratura di riferimento. Infine, la profondità massima di un senso in WordNet ne indica il livello di specificità.

Nella seguente cella si vanno a considerare tutti i synsets del dataset, creando una colonna corrispondente ad ogni feature discussa. La frequenza del termine viene calcolata rispetto ad un vocabolario messo a disposizione dalla libreria `wordfreq`. La lunghezza viene successivamente scalata in un intervallo $(0, 1)$ per permettere ai modelli di convergere più facilmente (in particolare modelli SVM).

In [ ]:
print('Installing dependecy...')
!pip install --quiet wordfreq
print('Done.')
from wordfreq import word_frequency
from sklearn.preprocessing import MinMaxScaler

# Create the aforementioned features
dataset['freq'] = [word_frequency(synset_name(synset), 'en') for synset in dataset['synsets']]
dataset['len'] = [len(synset_name(synset)) for synset in dataset['synsets']]
dataset['depth'] = [synset.max_depth() for synset in dataset['synsets']]

# Normalize lenght value
dataset['len'] = MinMaxScaler().fit_transform(dataset['len'].to_numpy().reshape(-1, 1))

dataset

Installing dependecy...
Done.


,synsets,is_basic,freq,len,depth
0,Synset('war.n.01'),1,2.880000e-04,0.000000,7
1,Synset('fiefdom.n.01'),0,2.000000e-07,0.153846,6
2,Synset('bed.n.03'),1,1.170000e-04,0.000000,5
3,Synset('return_on_invested_capital.n.01'),0,1.240000e-05,0.884615,6
4,Synset('texture.n.02'),1,8.510000e-06,0.153846,9
...,...,...,...,...,...
499,Synset('reading.n.03'),1,1.380000e-04,0.153846,6
500,Synset('sanctimoniousness.n.01'),0,1.150000e-08,0.538462,10
501,Synset('chalcedony.n.01'),0,1.200000e-07,0.269231,8
502,Synset('stopcock.n.01'),0,3.090000e-08,0.192308,11


Creiamo ora una batteria di modelli per fare classificazione e una funzione per testarli in *cross validation*. Successivamente utilizziamo tale funzione sul dataset creato in precedenza. Ovviamente i synsets in questo caso saranno rappresentati solamente dalle $2$ features significative discusse in precedenza.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Function to test a dataset over a series of models
def test_models(X, Y):
  # Define the models to test in a dictionary
  models = {
      'Logistic Regression': LogisticRegression(random_state = 42, max_iter = 500),
      'Support Vector Machine': SVC(kernel='linear', C=1, random_state = 42),
      'Decision Tree': DecisionTreeClassifier(),
  }
  # Iterate through each classifier and take its mean score over 5 folds
  for clf_name, clf in models.items():
    score = cross_val_score(clf, X, Y, cv = 5).mean()
    print(f'{clf_name}: {score}')

Una volta creata la funzione per testare i vari modelli scelti, creiamo il dataset con le features discusse in precedenza e richiamiamola su di esso.

In [ ]:
# Create the dataset by appending freq and len columns together
X = np.vstack([dataset['freq'], dataset['len'], dataset['depth']]).T
Y = dataset['is_basic']

test_models(X, Y)

Logistic Regression: 0.763960396039604
Support Vector Machine: 0.7758613861386139
Decision Tree: 0.8611683168316832


## BERT embeddings

Al posto di utilizzare delle features selezionate a mano, possiamo pensare ad un approccio alternativo che si basa sull'utilizzo di embeddings. L'idea è quella di trasformare un synset in un vettore numerico (embedding) tramite un modello transformer pre-trained. Sperabilmente questo vettore conterrà implicitamente delle informazioni riguardo alla basicness che potranno essere apprese eventualmente da uno dei classificatori definiti in precedenza.

Per semplicità di utilizzo, si è deciso di usare la libreria `flair` che mette a disposizione oggetti specifici per importare modelli transformers pretrained.

In [ ]:
print('Installing dependecy...')
!pip install -q flair
print('Done.')
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings, TransformerWordEmbeddings

Installing dependecy...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.1 MB/s eta 0:00:00
Done.


Una volta importata la libreria sono stati individuati due possibili approcci per ottenere un embedding a partire da un synset.

Il primo consiste nel creare un embedding a partire dalla *glossa* di un synset, mentre il secondo consiste nel creare un embedding per ogni *lemma* del synset, per poi farne la media.

Siccome non è possibile stabilire a priori quale delle due metodologie sia la migliore, sono state implementate entrambe per poterle valutare empiricamente.

In [ ]:
import torch

# Load pretrained transformer models
pretrained_model_name = 'bert-base-uncased'

sentence_embedder = TransformerDocumentEmbeddings(pretrained_model_name)
word_embedder = TransformerWordEmbeddings(pretrained_model_name)

# Get the embedding of the synset gloss
def embed_synset_gloss(synset):
  gloss = Sentence(synset.definition())
  sentence_embedder.embed(gloss)
  return gloss.embedding.numpy()

# Get the embeddings of each word that consitutes the synset, then return
# the average of them (https://randorithms.com/2020/11/17/Adding-Embeddings.html)
def embed_synset_lemmas(synset):
  lemmas = [lemma.replace('_', ' ') for lemma in synset.lemma_names()]
  lemmas = Sentence(lemmas)
  word_embedder.embed(lemmas)
  # Stack each token embedding
  embeddings = torch.stack([lemma.embedding for lemma in lemmas])
  # Get the mean of all token embeddings
  return torch.mean(embeddings, axis = 0).numpy()

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Una volta create le funzioni per ottenere a partire da un syset l'embedding corrispondente, non resta che applicare tali funzioni ad ogni synset.

**Nota bene**: l'esecuzione della seguente cella potrebbe richiedere un tempo considerevole dovuto all'operazione onerosa di embedding (la quale richiede il forward di una rete neurale di dimensioni notevoli).

In [ ]:
print('Computing gloss embeddings...')
dataset['gloss_embedding'] = dataset['synsets'].map(lambda syn: embed_synset_gloss(syn))
print('Computing lemmas embeddings...')
dataset['lemmas_embedding'] = dataset['synsets'].map(lambda syn: embed_synset_lemmas(syn))

Computing gloss embeddings...
Computing lemmas embeddings...


In [ ]:
# Evaluate Gloss embeddings
print('+=====+ Gloss Embeddings Scores +=====+')
X = np.vstack(dataset['gloss_embedding'])
test_models(X, Y)

# Evaluate Lemmas embeddings
X = np.vstack(dataset['lemmas_embedding'])
print('\n+=====+ Lemmas Embeddings Scores +=====+')
test_models(X, Y)


+=====+ Gloss Embeddings Scores +=====+
Logistic Regression: 0.6448316831683168
Support Vector Machine: 0.6150693069306931
Decision Tree: 0.5693465346534654

+=====+ Lemmas Embeddings Scores +=====+
Logistic Regression: 0.7857029702970297
Support Vector Machine: 0.7420990099009901
Decision Tree: 0.6288910891089109


## Conclusioni
Possiamo vedere come in media le feature estratte a mano sembrano essere più adatte alla classificazione. Questo potrebbe essere dato dal fatto che gli embeddings di questi modelli transformers pre-trained potrebbero non aver catturato al loro interno le informazioni che possono essere utili riguardo alla *basicness* di un synset.

D'altra parte, un'altra problematica potrebbe essere quella dell'assenza dei dati. In generale, solamente $500$ annotazioni potrebbero non essere sufficienti ad apprendere le dipendenze statistiche discriminative da parte di modelli di machine learning.